# Part 1 : Solve a system of euqations

In [ ]:
# equations:
#1： x + y - z = 0
#2：2x - y + z = 3
#3: 3x + y +2z = 11

# Ax|b 

# 1 1 -1 | 0
# 2 -1 1 | 3
# 3 1 2  | 11

#Result: x = 1, y = 2 , z = 3

# Part 2: Write a function and solve using elimination

In [87]:
import numpy as np 
from scipy import linalg 
import math 


In [23]:
def linearsolver(A,b):   
    n = len(A)+1 
    M = A 

    i = 0 
    for x in M:
        x.append(b[i]) 
        i += 1 

    for k in range(n):   
        for i in range(k,n): 
            if abs(M[i][k]) > abs(M[k][k]):  
                M[k], M[i] = M[i],M[k]  
            else: 
                pass  

    for j in range(k+1,n):
        q = M[j][k] / M[k][k] 
        for m in range(k, n+1):         
            M[j][m] +=  q * M[k][m]

    x = [0 for i in range(n)]

    x[n] =float(M[n][n+1])/M[n][n]
    for i in range (n-1,-1,-1): 
        z = 0 
        for j in range(i+1,n):
            z = z  + float(M[i][j])*x[j] 
        x[i] = float(M[i][n+1] - z)/M[i][i]
    print(x)

In [25]:
A = np.array([[1, 4, 4], [2, 5, 3], [2, 7, 1]])
b = np.array([5, 5, 4])
linearsolver(A,b)


[0.5 0.3125 0.8125]


# Part 3. 

### Show that $AA^T$ does not equal to $A^TA$ in general
Assume A with shape of (x,y)
For $AA^T$, is a y * x matrix multiplies a x * y matric results a y * y matrix
For $A^TA$, is a x * y matrix multiplies a y * x matric results a x * x matrix
Under most situation, y * y is not equal to x * x

### When  $AA^T$ does equal to $A^TA$
When A is symetric, $A==A^T$, this equation is right. 

# Part 4: Matrix Factorization

In [94]:


def mult_matrix(M, N):
     #conver N                                                                                                                                                                                               
    tuple_N = zip(*N)
                                                                                                                                                                                 
    return [[sum(el_m * el_n for el_m, el_n in zip(row_m, col_n)) for col_n in tuple_N] for row_m in M]

#identy matrix
def pivot_matrix(M):
    m = len(M)

                                                                                                                                                                                          
    id_mat = [[float(i ==j) for i in xrange(m)] for j in xrange(m)]
                                                                                                                                                                                  
                                                                                                                                                                                              
    for j in xrange(m):
        row = max(xrange(j, m), key=lambda i: abs(M[i][j]))
        if j != row:
                                                                                                                                                                                                                                       
            id_mat[j], id_mat[row] = id_mat[row], id_mat[j]

    return id_mat

def lu_decomposition(A):

    n = len(A)

    # Create L and U                                                                                                                                                                                                                 
    L = [[0.0] * n for i in xrange(n)]
    U = [[0.0] * n for i in xrange(n)]

                                                                                                                                                                                              
    P = pivot_matrix(A)
    PA = mult_matrix(P, A)

    #decomposition                                                                                                                                                                                                                 
    for j in xrange(n):
        # All diagonal entries of L are set to unity                                                                                                                                                                                                   
        L[j][j] = 1.0

                                                                                                                                                                                     
        for i in xrange(j+1):
            s1 = sum(U[k][j] * L[i][k] for k in xrange(i))
            U[i][j] = PA[i][j] - s1

                                                                                                                                                                  
        for i in xrange(j, n):
            s2 = sum(U[k][j] * L[i][k] for k in xrange(j))
            L[i][j] = (PA[i][j] - s2) / U[j][j]

    return (L, U)



In [98]:

A = np.array([5, 2, 5], [1, 4, 5], [1, 2, 15])
L, U = lu_decomposition(A)

print "A:"
print(A)

print "L:"
print(L)

print "U:"
pprint.pprint(U)



A:
[[ 5.  2.  5.]
 [ 1.  4.  5.]
 [ 1.  2. 15.]]
L:
[[1.         0.         0.        ]
 [0.2        1.         0.        ]
 [0.2        0.45       1.        ]]
U:
[[ 5.        2.        5.      ]
 [ 0.        3.6       4.      ]
 [ 0.        0.       12.]]


# Part 5: Build your own function to solve a determinant for any matrix

In [49]:
def dtmn(A):
    #special situation
    if A.shape[0] == 1:
        return A[0][0]
    
    D = 0
    for i in range(A.shape[0]):
        A1 = np.delete(A,0,0)
        A1 = np.delete(A1,i,1)
        D += (-1)**(1+i+1)*A[0,i]*dtmn(A1) 
    
    return D

In [106]:
# TEST 2X2
x = np.array([[2,4], [1,3]])
print("dtmn:", dtmn(x))
print("Real:", np.linalg.det(x))

dtmn: 2
Real: 2.0


In [107]:
# TEST 3X3 
x = np.array([[1,4,6], [5,1,7],[2,4,3]])
print("dtmn:", dtmn(x))
print("Real:", np.linalg.det(x))

dtmn: 79
Real: 79.0


In [108]:
# TEST 4X4
x = np.array([[2,3,5,2], [1,2,-3,6],[-9,8,2,3],[5,-3,1,2]])
print("dtmn:", dtmn(x))
print("Real:", np.linalg.det(x))

dtmn: 841
Real: 841.0000000000002


# Part 6: Build your own function from scratch to solve equations via Cramer’s rules.

In [74]:
def cmr(A, b):
    m = A.shape[0]
    b1 = b.reshape((m, 1))
    x = np.zeros(m)
    detA = dtmn(A)
    
    for i in range(0, m):
        n = np.concatenate((A[0:m:1, 0:i:1], b1), axis=1)
        n = np.concatenate((n, A[0:m:1, i+1:m:1]), axis=1)
        x[i] = dtmn(n) / detA
    
    return x

In [81]:
# TEST 2X2
A = np.array([[2, 3], [1, 4]])
b = np.array([6, 8])
print("cmr:", cmr(A, b))
print("Real: ", np.linalg.solve(A, b))

cmr: [0. 2.]
Real:  [0. 2.]


In [80]:
# TEST 3X3
A = np.array([[1, 3, 3], [2, 5, 1], [6, 2, 2]])
b = np.array([6, 4, 6])
print("cmr:", cmr(A, b))
print("Real: ", np.linalg.solve(A, b))

cmr: [0.375   0.34375 1.53125]
Real:  [0.375   0.34375 1.53125]


In [75]:
# TEST 4X4
A = np.array([[5, 3, 4, 2], [3, 1, 4, 9], [3, 5, 6, 4], [2, 6, 5, 1]])
b = np.array([5, 8, 22, 3])
print("cmr:", cmr(A, b))
print("Real: ", np.linalg.solve(A, b))

cmr: [ -6.91964286 -19.20535714  27.82142857  -7.03571429]
Real:  [ -6.91964286 -19.20535714  27.82142857  -7.03571429]
